In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [8]:
df = pd.read_csv('teste_indicium_precificacao.csv')


In [9]:
df = df.drop(columns=['id', 'nome', 'host_id', 'host_name', 'ultima_review', 'reviews_por_mes', 'latitude', 'longitude', 'bairro', 'calculado_host_listings_count'])
df.head()

,bairro_group,room_type,price,minimo_noites,numero_de_reviews,disponibilidade_365
0,Manhattan,Entire home/apt,225,1,45,355
1,Manhattan,Private room,150,3,0,365
2,Brooklyn,Entire home/apt,89,1,270,194
3,Manhattan,Entire home/apt,80,10,9,0
4,Manhattan,Entire home/apt,200,3,74,129


In [10]:
colunas_outliers = ['price', 'minimo_noites', 'numero_de_reviews', 'disponibilidade_365']

#função que remove outliers com IQR
def remover_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)  
    Q3 = df[col].quantile(0.75)  
    IQR = Q3 - Q1 
    lower_bound = Q1 - 1.5 * IQR  
    upper_bound = Q3 + 1.5 * IQR  
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]


for coluna in colunas_outliers:
    df = remover_outliers_iqr(df, coluna)

print(df.describe())


              price  minimo_noites  numero_de_reviews  disponibilidade_365
count  35201.000000   35201.000000       35201.000000         35201.000000
mean     119.288799       2.765092          12.362632            87.923781
std       67.832345       1.917478          16.631702           122.289098
min        0.000000       1.000000           0.000000             0.000000
25%       65.000000       1.000000           1.000000             0.000000
50%      100.000000       2.000000           5.000000            11.000000
75%      155.000000       3.000000          17.000000           158.000000
max      334.000000      11.000000          71.000000           365.000000


In [12]:
X = df.drop(columns=['price'])
y = df['price']


In [13]:
X = pd.get_dummies(X, dtype=int)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

In [15]:
model = RandomForestRegressor(random_state=42)

In [16]:

#hiperparam. p teste
param_grid = {
    'n_estimators': [50, 100, 200, 300],  
    'max_depth': [5, 10, 15, None],  
    'max_features': [None, 'sqrt', 'log2']  
}


In [17]:
#busca dos melhores valores
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

In [18]:

grid_search.fit(X_train, y_train)

print("melhores parâmetros:", grid_search.best_params_)

melhores parâmetros: {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 300}


In [19]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [20]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error: 36.45930325291385
Mean Squared Error: 2454.97567782858
R² Score: 0.4744


In [21]:
#predição do modelo apresentado


apartamento = {
    'id': 2595,
    'nome': 'Skylit Midtown Castle',
    'host_id': 2845,
    'host_name': 'Jennifer',
    'bairro_group': 'Manhattan',
    'bairro': 'Midtown',
    'latitude': 40.75362,
    'longitude': -73.98377,
    'room_type': 'Entire home/apt',
    'minimo_noites': 1,
    'numero_de_reviews': 45,
    'ultima_review': '2019-05-21',
    'reviews_por_mes': 0.38,
    'calculado_host_listings_count': 2,
    'disponibilidade_365': 355
}


apartamento_df = pd.DataFrame([apartamento])


apartamento_df = apartamento_df.drop(columns=['id', 'nome', 'host_id', 'host_name', 'ultima_review', 'reviews_por_mes', 'latitude', 'longitude', 'bairro', 'calculado_host_listings_count'])


apartamento_df = pd.get_dummies(apartamento_df, dtype=int)


apartamento_df = apartamento_df.reindex(columns=X.columns, fill_value=0)


previsao = best_model.predict(apartamento_df)

print(f"Previsão de preço para o apartamento: ${previsao[0]:.2f}")

Previsão de preço para o apartamento: $202.39
